# nba-rating

Expected score of Team A:
$$E_A=\frac{1}{1+10^{(R_B-R_A)/400}}$$

Expected score of Team B:
$$E_B=\frac{1}{1+10^{(R_A-R_B)/400}}$$

These are also represented by: 
$$E_A=\frac{Q_A}{Q_A+Q_B}$$
$$E_B=\frac{Q_B}{Q_A+Q_B}$$


$Q_A=10^{R_A/400}$, $Q_B=10^{R_B/400}$

In [28]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
from datetime import datetime
import requests
import bs4
from NBARater import NBARater

In [29]:
# Script for scraping data
!python NBAScraper.py --beginning 2013 --end 2024

Scraping data for the 2013 to 2023 seasons...
Fetching 2013-2014 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2014-2015 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2015-2016 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2016-2017 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2017-2018 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2018-2019 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2019-2020 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2020-2021 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2021-2022 season...
Loading: 100%|███████████████████████████████

In [30]:
df = pd.read_csv(Path('data')/'2012-2023.csv')
df

,Date,Start Time (ET),Visitor,Visitor Points,Home,Home Points,Box Score,Overtime,Attendance,Arena,Notes
0,2012-10-30,7:00p,Washington Wizards,84,Cleveland Cavaliers,94,https://www.basketball-reference.com//boxscore...,NaN,"20,562",Quicken Loans Arena,NaN
1,2012-10-30,8:00p,Boston Celtics,107,Miami Heat,120,https://www.basketball-reference.com//boxscore...,NaN,"20,296",AmericanAirlines Arena,NaN
2,2012-10-30,10:30p,Dallas Mavericks,99,Los Angeles Lakers,91,https://www.basketball-reference.com//boxscore...,NaN,"18,997",STAPLES Center,NaN
3,2012-10-31,7:00p,Denver Nuggets,75,Philadelphia 76ers,84,https://www.basketball-reference.com//boxscore...,NaN,"19,101",Wells Fargo Center,NaN
4,2012-10-31,7:00p,Indiana Pacers,90,Toronto Raptors,88,https://www.basketball-reference.com//boxscore...,NaN,"19,800",Air Canada Centre,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12931,2023-06-01,8:30p,Miami Heat,93,Denver Nuggets,104,https://www.basketball-reference.com//boxscore...,NaN,"19,528",Ball Arena,Playoffs
12932,2023-06-04,8:00p,Miami Heat,111,Denver Nuggets,108,https://www.basketball-reference.com//boxscore...,NaN,"19,537",Ball Arena,Playoffs
12933,2023-06-07,8:30p,Denver Nuggets,109,Miami Heat,94,https://www.basketball-reference.com//boxscore...,NaN,"20,019",Kaseya Center,Playoffs
12934,2023-06-09,8:30p,Denver Nuggets,108,Miami Heat,95,https://www.basketball-reference.com//boxscore...,NaN,"20,184",Kaseya Center,Playoffs


In [31]:
df['Notes'].value_counts()

Playoffs                  535
Play-In Game               18
at Mexico City, Mexico     10
at London, England          7
at Paris, France            1
Name: Notes, dtype: int64

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Win'] = np.where(df['Visitor Points'] > df['Home Points'], 1, np.where(df['Visitor Points'] < df['Home Points'], 0, 0.5))
df

In [ ]:
rating = NBARater()
rating.eloSimulator(df)

In [ ]:
teams_elo = rating.getTeams()

In [ ]:
# Create traces for each key
traces = []
for key, values in teams_elo.items():
    trace = go.Scatter(x=list(range(1, len(values) + 1)), y=values, mode='lines+markers', name=key)
    traces.append(trace)

# Create layout
layout = go.Layout(title='Interactive Line Plot', xaxis=dict(title='Index'), yaxis=dict(title='Values'))

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show the interactive plot
fig.show()

In [ ]:
latest_rating = {}
for key, value in teams_elo.items():
    latest_rating[key] = value[-1]
dict(sorted(latest_rating.items(), key=lambda item: item[1], reverse=True))

In [ ]:
import requests
import bs4

url = f'https://en.wikipedia.org/wiki/2020_NBA_playoffs'
response = requests.get(url)
response.raise_for_status()

soup = bs4.BeautifulSoup(response.text, 'lxml')
dates = (soup.find('body')
         .find('div', class_='mw-page-container')
         .find('div', class_='mw-page-container-inner')
         .find('div', class_='mw-content-container')
         .find('main', class_='mw-body')
         .find('div', class_='mw-body-content')
         .find('div', class_='mw-content-ltr mw-parser-output')
         .find('table', class_='infobox vcard')
         .find('tbody')
         .find_all('tr')[3]
         .find('td', class_='infobox-data')).text
dates

In [ ]:
date = dates.split(',')
date

In [ ]:
date[0].split('–')[0].strip()